In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

spark

In [ ]:
json_schema = StructType([
    StructField("Order_id", StringType(), False),  
    StructField("Name", StringType(), False),
    StructField("Email", StringType(), False),  
    StructField("latitude", LongType(), True),
    StructField("longitude", LongType(), True),
    StructField("created_at", LongType(), True)
])
data_df = spark.createDataFrame([], json_schema)

data_df.printSchema()

In [ ]:
host = "localhost"
port = 8888

lines = spark.readStream \
    .format("socket") \
    .option("host", host) \
    .option("port", port) \
    .load()

lines.isStreaming()
lines.printSchema()

query = lines.writeStream \
    .outputMode("append") \
    .foreach(lambda row: print(row.value)).start()
query.awaitTermination()
spark.stop()